In [ ]:
# NaN = 1 to drop NaN values in dataframe, else NaN = 0
NaN = 1

# correlated = 1 to drop correlated features in dataframe, else correlated = 0
correlated = 1

# correlation lower bound to drop features
corr_bound = 0.91
# minimum difference in IV to drop feature based on correlation bound
iv_diff = 0.1 #Feat_sum.loc[:,"Total_IV"].std()

In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import os.path
import rpy2
import scorecardpy as sc
from scorecardpy import info_value 
import seaborn as seabornInstance 
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn import feature_selection
from sklearn.feature_selection import RFE, RFECV, mutual_info_classif, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier as RFE
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC

In [ ]:
f = os.path.expanduser( "~/Documents/Vodacom/data/df_raw_final.pkl" )

df = pd.read_pickle(f)

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
#new dataframe after dropping all values with NaN
dfclean = df.dropna( subset = ['target_label'] )

In [ ]:
#replace all strings "yes", "no", "maybe" with integers 0, 1, 2 
dfclean = dfclean.replace( to_replace = "yes", value = 1 )
dfclean = dfclean.replace( to_replace = "no", value = 0 )
dfclean = dfclean.replace( to_replace = "maybe", value = 2 )

In [ ]:
# remove all rows with target_label = 2 ("maybe")
dfclean = dfclean[ dfclean.target_label != 2 ]

In [ ]:
# remove all columns that are targets 
for feature in dfclean :
    
    if (dfclean[feature].isna().sum() > 100) :
        dfclean = dfclean.drop( columns = feature )

Column 'geometry' in dataframe causes unwanted errors.
Initialize new dataframe without the 'geometry' column.
Assumption that the 'geometry' column causes more issues than has relevance for model.

In [ ]:
# remove column geometry due to string error with conversion
dfclean_nogeo = dfclean.drop( columns = "geometry")

# add randomized feature
# dfclean_nogeo['randfeature'] = np.random.normal(size = dfclean_nogeo.shape[0])

Creating lists of numerical and categorical features

In [ ]:
Feat_typ = {}
Num_typ = {}

num_list = []
cat_list = []
index_list = []

j = 1

# for loop to identify feature types

for i, feature in enumerate( dfclean_nogeo ):
    
    if ( dfclean_nogeo[ feature ].dtype == 'float64' or dfclean_nogeo[ feature ].dtype == 'int64'):
        
        value_list = dfclean_nogeo[feature].values
        length_values_list = []
        
# for loop identifying if all items in a given feature have the same number of characters
# if they do, identify feature as a category
        
        for item in value_list:
            
            length_values_list.append( len( str( item ) ) )

# CODE = true if number of characters in values of all features are the same            
            
        CODE = ( len( set( length_values_list ) ) == 1)
        
    if (dfclean_nogeo[feature].dtype == 'object' or CODE):     

        Feat_typ[feature] = [i,"cat"]
        index_list.append(feature)
        if (feature != 'target_label') :
            
            cat_list.append(feature)
    
    else:
        
        Feat_typ[feature] = [i, "num"]
        Num_typ[feature] = [j, "num"]
        num_list.append(feature)
        index_list.append(feature)
        j = j + 1
    
    # adding the 'target_label' column to num_list    
    
    if feature == "target_label":

        num_list.append( feature )


Activates the use of R in the Jupyter Notebook

In [ ]:
%load_ext rpy2.ipython

Brute force fix for kernel dying on Mac, Error15

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
%%R -i dfclean_nogeo -i num_list -o cuts

library(smbinning)
library(foreach)
library(doParallel)


comb <- function(...){
    mapply('cbind', ..., SIMPLIFY = FALSE)
}

no_cores <- detectCores()
registerDoParallel(cores = 4)

numg_list = c()
cutsgood = c()

Features <- names(dfclean_nogeo)[1:ncol(dfclean_nogeo)]

i = 1
j = 1

ptime <- system.time({

 cuts = foreach( col = Features, .combine = comb, .packages = 'smbinning')  %dopar%  {

    sbin = smbinning( dfclean_nogeo, 'target_label', col )
            
       if ( !is.na( sbin[ 'bands' ] ) ) {
        
            cutsgood[[i]] <- sbin$bands[ -length( sbin$bands ) ]
            numg_list[i] = col
            i = i + 1
           
        } 
     
        # adding target_label to numg_list
        if ( col == "target_label" ) {

            numg_list[i] = col

        }
     
     return(list(numg_list, cutsgood))
     
    }
    
})[3]
print(paste("Time taken for smbinning =", ptime, "seconds"))

In [ ]:
# create a list for numg_list from an R vector
cuts = list(cuts)

# creating dictionary of value cutsgood
cutsg_dic = dict( zip( cuts[0], cuts[1] ) )

The woebin function creates IV tables for all numerical features in dataframe. It uses smbinning's cut values for the bins for all numerical features within cutsg_dic, all other numerical features have bins cut by the woebin function. 
This is due to the assumption that smbinning's cut values for the bins will yielf a maximized IV while, woebinning's cut values will not be maximized.

In [ ]:
# remove all columns that are targets or categories

dfclean_nogeo = dfclean_nogeo.drop( columns = cat_list )

In [ ]:
# information values of all numerical features
num_iv = sc.woebin( dfclean_nogeo.loc[ :, num_list ],
                   y = "target_label", breaks_list = cutsg_dic, positive = 0, bin_num_limit = 20 )

In [ ]:
# new dataframe summarizing features
Feat_sum = pd.DataFrame( dfclean_nogeo.dtypes )

i = 0

for feature in dfclean_nogeo:
    
    Feat_sum.loc[ feature,"Unique" ] = dfclean_nogeo[ feature ].nunique()
    Feat_sum.loc[ feature, "NaNs" ] = dfclean_nogeo[ feature ].isna().sum()
    i = i + 1

Commented out code that follows is for the use of all and categorical features. Decision was made to use only Numerical features. 

In [ ]:
# all_iv = sc.woebin( dfclean_nogeo.loc[ :, index_list ],
#                    y = "target_label", breaks_list = cutsg_dic, positive = 0, bin_num_limit = 20 )

In [ ]:
# IV values of all categorical features in dataframe
#cat_iv = sc.info_value.iv( dfclean_nogeo.loc[ :, cat_list ], y = "target_label" )

In [ ]:
# # changing the headings of the dataframe cat_iv
# cat_iv = cat_iv.rename( columns = { 'variable':'', 'info_value': 'Total_IV' } )

# cat_iv = cat_iv.set_index('')

In [ ]:
# # adding iv values of numerical features to summary dataframe
# for feature in dfclean_nogeo.loc[ :, num_list ]:
    
#     if ( feature != 'target_label') :
        
#         Feat_sum.loc[ feature,'Total_IV' ] = num_iv[ feature ].total_iv[ 1 ]

In [ ]:
# # adding iv values of categorical features to summary datframe
# for feature in dfclean_nogeo.loc[ :, cat_list ]:
    
#     if ( feature != 'target_label' ) :
        
#         Feat_sum.loc[ feature,'Total_IV' ] = cat_iv.loc[ feature, 'Total_IV' ]
        

In [ ]:
# adding iv values of numerical features to summary dataframe
for feature in dfclean_nogeo.loc[ :, num_list ]:
    
    if ( feature != 'target_label' ) :
        
        Feat_sum.loc[ feature,'Total_IV' ] = num_iv[ feature ].total_iv[ 1 ]
        

In [ ]:
# with open('Feat_sum.data', rb) as filehandle :
# Feat_sum.to_csv('Feat_sum.csv')
# df_final.to_csv('df_final.csv')

In [ ]:
# Feat_sum.sort_values('Total_IV', ascending = False).head(10)

In [ ]:
# creating correlation matrix
cor = dfclean_nogeo.corr()

# cor.style.background_gradient(cmap = 'coolwarm')

# joining Feat_sum matrix and correlation matrix
cor = cor.join(Feat_sum.Total_IV, how = 'outer')

In [ ]:
rem_feat = []
check_feat = []

for feature, correl in cor.iteritems():
    
    #for each feature
    for i in np.arange(len(correl.values)):
    
    # run through each row of cor
        
        if (abs(correl[i]) > corr_bound and correl.index[i] != feature and feature != "Total_IV"):
    #if the correlation is high and less than one and ignore total IV
            
            if (cor.loc[correl.index[i], "Total_IV"] - cor.loc[feature, "Total_IV"] > iv_diff):
    #if the IV of the row feature is more than 0.1 greater than the IV of the column feature, add the comun to the remove list
#                 print("row ",correl.index[i],"   is bigger than column     ", feature)
                rem_feat.append(feature)
            
            elif(cor.loc[correl.index[i], "Total_IV"] - cor.loc[feature, "Total_IV"] > 0):
    #if the features have very similar IVs, add them to the check list
                check_feat.append([feature, correl.index[i]])

rem_feat = list(dict.fromkeys(rem_feat))     #original order

# rem_feat =  list(set(rem_feat))            #fastest order
# rem_feat =  set(rem_feat)                  #alphabetical order
# rem_feat =  rem_feat                       #with duplicates


The next kernel refers to the first kernel of the notebook. It is used so simplify the experimental process to identify optimal feature selection.

In [ ]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)

# drop all rows containing NaN values 
if (NaN == 1):
    
    df_final_noNaN = dfclean_nogeo.dropna()
    
else:
        
    df_final_noNaN = dfclean_nogeo

In [ ]:
# drop columns based off of correlation comparison
if (correlated == 1) :
    
    df_final_noNaN = df_final_noNaN.drop(columns = rem_feat)

In [ ]:
#cat_woe = sc.woebin( dfclean_nogeo.loc[ :, cat_list ], y = "target_label", bin_num_limit = 20 )
#df_final2 = sc.woebin_ply(df_final, cat_woe)

In [ ]:
# transforming data table to WOE values

#df_final = sc.woebin_ply(df_final_noNaN, num_iv)
df_final = df_final_noNaN

# IV list maximization


In [ ]:
# for i, feat in enumerate(Feat_sum.iterrows()):
#     Feat_sum = Feat_sum.rename(index = {feat[0]: feat[0] + "_woe"})

In [ ]:
Feat_sum = Feat_sum.loc[df_final.columns]
Feat_sum = Feat_sum.sort_values("Total_IV", ascending = False)
Feat_sum.drop(Feat_sum.tail(1).index, inplace=True)

In [ ]:
no_feat_acc_iv = []
score2 = 0
nom_feat = 1
for no_feat in np.arange(len(Feat_sum)):
    
    used_feat = list(Feat_sum[0:no_feat+1].index)
    df_final_X = df_final.loc[:,used_feat]
    df_final_Y = pd.DataFrame(df_final.target_label)
    X_train_iv, X_test_iv, y_train_iv, y_test_iv = train_test_split(df_final_X, df_final_Y,
                                                       test_size = 0.3,stratify = df_final_Y,
                                                       random_state = 40)
    logreg = LogisticRegression()
    logreg.fit(X_train_iv, y_train_iv)
    y_pred_iv = logreg.predict(X_test_iv)
    score1 = logreg.score(X_test_iv, y_test_iv)
    if (score1 > score2) :
        score2 = score1
        nom_feat = nom_feat + 1
        best_feat = list(Feat_sum[0:nom_feat].index)
        no_feat_acc_iv.append([logreg.score(X_test_iv, y_test_iv), nom_feat, best_feat])
    

In [ ]:
# 0.889589905362776, 56 when no correlation
# 0.886435331230284, 77 when correlation dropped at 0.92
# 0.8911671924290221, 52 when correlation dropped at 0.9
# 0.8801261829652997, 43 when correlation dropped at 0.8

In [ ]:
# 0.889589905362776, 11  when no correlation
# 0.8911671924290221, 10  when corr dropped at 0.91
# 0.886435331230284, 9  when corr dropped at 0.92
# 0.8927444794952681, 10  when corr dropped at 0.85

In [ ]:
max(no_feat_acc_iv)

In [ ]:
# clf = SVC(kernel='linear', C=1).fit(X_train_iv, y_train_iv)
# clf.score(X_test_iv, y_test_iv)

In [ ]:
# from sklearn.model_selection import cross_val_score

# clf = SVC(kernel='linear', C=1)
# scores = cross_val_score(clf, X_test_iv, y_test_iv, cv=5)
# scores

# WOE and not WOE 

In [ ]:
df_final_x_woe = df_final.drop(["target_label"], axis = 1)
df_final_y_woe = pd.DataFrame(df_final.target_label)

df_final_x = df_final_noNaN.drop(["target_label"], axis = 1)
df_final_y = pd.DataFrame(df_final_noNaN.target_label)

In [ ]:
X_train_woe, X_test_woe, y_train_woe, y_test_woe = train_test_split(df_final_x_woe, df_final_y_woe, 
                                                test_size = 0.3, stratify = df_final_y_woe,
                                                random_state = 40)

X_train, X_test, y_train, y_test = train_test_split(df_final_x, df_final_y, 
                                                test_size = 0.3, stratify = df_final_y,
                                                random_state = 40)

In [ ]:
regressor_woe = LogisticRegression()  
regressor_woe.fit(X_train_woe, y_train_woe)
  
y_pred_woe = regressor_woe.predict(X_test_woe)

In [ ]:
regressor = LogisticRegression()  
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

In [ ]:
select_feature_woe = mutual_info_classif(X_train_woe, y_train_woe)

select_feature = mutual_info_classif(X_train, y_train)

In [ ]:
selected_features_df_woe = pd.DataFrame({'Features': list(X_train_woe.columns),
                                     'Scores': select_feature_woe})
selected_features_df_woe = selected_features_df_woe.sort_values(by = 'Scores', ascending = False)

selected_features_df = pd.DataFrame({'Features': list(X_train.columns),
                                     'Scores': select_feature})
selected_features_df = selected_features_df.sort_values(by = 'Scores', ascending = False)

# Mutual Information

In [ ]:
no_feat_acc_mi = []
score2 = 0
nom_feat = 1
for no_feat in np.arange(len(selected_features_df_woe)):
    
    used_feat = list(selected_features_df_woe.iloc[0:no_feat+1,0])
    df_final_X = df_final.loc[:,used_feat]
    df_final_Y = pd.DataFrame(df_final.target_label)
    X_train_mi, X_test_mi, y_train_mi, y_test_mi = train_test_split(df_final_X, df_final_Y,
                                                       test_size = 0.3,stratify = df_final_Y,
                                                       random_state = 40)
    logreg = LogisticRegression()
    logreg.fit(X_train_mi, y_train_mi)
    y_pred = logreg.predict(X_test_mi)
    score1 = logreg.score(X_test_mi, y_test)
    if (score1 > score2) :
        score2 = score1
        nom_feat = nom_feat + 1
        best_feat = list(selected_features_df_woe[0:nom_feat].Features)
        no_feat_acc_mi.append([logreg.score(X_test_mi, y_test_mi), nom_feat, best_feat])
    

In [ ]:
# 0.88801261829653, 58 with nothing removed
# 0.891167, 62 when correlation dropped at 0.92
# 0.889589905362776, 49 when correlation dropped at 0.9
# 0.8801261829652997, 46 when correlation dropped at 0.8
# 0.8943217665615142, 13 when correlation dropped at 0.92

In [ ]:
# 0.8927444794952681, 17 withh correlation not dropped
# 0.8832807570977917, 11 withh corr dropped at 0.91
# 0.889589905362776, 8 withh corr dropped at 0.92
# 0.8911671924290221, 10 withh corr dropped at 0.85

In [ ]:
max(no_feat_acc_mi)

In [ ]:
confusion_matrix_woe = metrics.confusion_matrix(y_test_woe, y_pred_woe)
print(confusion_matrix_woe)

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(confusion_matrix)

In [ ]:
score_woe = regressor_woe.score(X_test_woe, y_test_woe)
print(score_woe)

In [ ]:
score = regressor.score(X_test, y_test)
print(score)

In [ ]:
print(classification_report(y_test_woe, y_pred_woe))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
# logit_roc_auc_woe = roc_auc_score(y_test_woe, regressor_woe.predict(X_test_woe))
# fpr, tpr, thresholds = roc_curve(y_test_woe, regressor_woe.predict_proba(X_test_woe)[:,1])
# plt.figure()
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc_woe)
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic')
# plt.legend(loc="lower right")
# plt.savefig('Log_ROC')
# plt.show()

In [ ]:
# logit_roc_auc = roc_auc_score(y_test, regressor.predict(X_test))
# fpr, tpr, thresholds = roc_curve(y_test, regressor.predict_proba(X_test)[:,1])
# plt.figure()
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic')
# plt.legend(loc="lower right")
# plt.savefig('Log_ROC')
# plt.show()

In [ ]:
# rfe_woe = RFE( random_state = 101 )
svc_woe = SVC(kernel="linear")
rfecv_woe = RFECV( estimator = svc_woe, step = 1, cv = StratifiedKFold(10), scoring = 'accuracy', n_jobs = 4 )
rfecv_woe.fit( X_test_woe, y_test_woe )

In [ ]:
rfe = RFE( random_state = 101 )
#svc = SVC(kernel="linear")
rfecv = RFECV( estimator = rfe, step = 1, cv = StratifiedKFold(10), scoring = 'accuracy', n_jobs = 4 )
rfecv.fit( X_test, y_test )

In [ ]:
# WOE transformation

# 0.8753943217665615, 30 with correlation not dropped
# 0.8785488958990536, 37 with corr dropped at 0.91
# 0.886435331230284, 30 with corr dropped at 0.92

In [ ]:
if (correlated == 1):
    print('Highly correlated features removed')
print('Optimal number of features with woe: {}'.format(rfecv_woe.n_features_))
print('Out of total: ', selected_features_df_woe.Features.nunique(), ' features')
print('Score', ' = ' ,score_woe)

In [ ]:
optimal_features_df_woe = pd.DataFrame({'Features': list(X_train_woe.columns),
                                     'Scores': rfecv_woe.support_})
#optimal_features_df_woe.loc[optimal_features_df_woe['Scores'] == True]

In [ ]:
# no WOE transformation

# 0.7886435331230284, 39 with correlation not dropped
# 0.7854889589905363, 17 with corr dropped at 0.91
# 0.8012618296529969, 69 with corr dropped at 0.92

In [ ]:
if (correlated == 1):
    print('Highly correlated features removed')
print('Optimal number of features: {}'.format(rfecv.n_features_))
print('Out of total: ', selected_features_df.Features.nunique(), ' features')
print('Score', ' = ' ,score)

In [ ]:
optimal_features_df = pd.DataFrame({'Features': list(X_train.columns),
                                     'Scores': rfecv.support_})
#optimal_features_df.loc[optimal_features_df['Scores'] == True]

In [ ]:
# plt.figure(figsize=(16, 9))
# plt.title('Recursive Feature Elimination WOE with Cross-Validation', fontsize=18, fontweight='bold', pad=20)
# plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
# plt.ylabel('% Correct Classification', fontsize=14, labelpad=20)
# plt.plot(range(1, len(rfecv_woe.grid_scores_) + 1), rfecv_woe.grid_scores_, color='#303F9F', linewidth=3)

# plt.show()

In [ ]:
# plt.figure(figsize=(16, 9))
# plt.title('Recursive Feature Elimination with Cross-Validation', fontsize=18, fontweight='bold', pad=20)
# plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
# plt.ylabel('% Correct Classification', fontsize=14, labelpad=20)
# plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, color='#303F9F', linewidth=3)

# plt.show()

In [ ]:
print(np.where(rfecv.support_ == False)[0])

In [ ]:
print('Optimal number of features: {}'.format(rfecv.n_features_))

In [ ]:
# IV
max(no_feat_acc_iv)

In [ ]:
# Mutual Information
max(no_feat_acc_mi)